In [1]:
import importlib
import optuna

In [2]:
from analysis_functions import *

/Users/i.grebenkin/.local/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/i.grebenkin/.local/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
importlib.reload(sys.modules['analysis_functions'])
from analysis_functions import *

In [4]:
X_stress_components_new = opener('X_stress_components_new', path_import='./resourses/')
X_strain_components_new = opener('X_strain_components_new', path_import='./resourses/')
y_stress_components_new = opener('y_stress_components_new', path_import='./resourses/')
y_strain_components_new = opener('y_strain_components_new', path_import='./resourses/')
# X_stress_components_other = opener('X_stress_components_other')
# X_strain_components_other = opener('X_strain_components_other')
# y_stress_components_other = opener('y_stress_components_other')
# y_strain_components_other = opener('y_strain_components_other')

./resourses//X_stress_components_new.pkl
./resourses//X_strain_components_new.pkl
./resourses//y_stress_components_new.pkl
./resourses//y_strain_components_new.pkl


In [5]:
X_stress_components_new.shape

(3, 2678, 5)

In [6]:
component_num = 0
n_trials_list = [10, 50, 100, 200, 500]
n_trials_list = [10, 50, 100, ]

In [7]:
best_params_list = []

In [ ]:
for n_trials in n_trials_list:
    best_params, cur_X_test, cur_y_test = do_optuna(X_stress_components_new[component_num],
                                                    y_stress_components_new[component_num],
                                                    n_trials=n_trials, n_layers=4, n_neurons=100)
    best_params_list.append(best_params)

In [ ]:
path_to_save_model = './learned_models/'
test_metrics = []
for i, best_params in enumerate(best_params_list):
    optuna_200_3cv, cur_prediction, cur_y_test, cur_X_test, test_metric = test_after_opt(
        best_params, X_stress_components[component_num], y_stress_components[component_num],
        f'optuna_{n_trials_list[i]}_3cv', path_to_save_model)
    test_metrics.append(test_metric)

In [60]:
sns.set_style('whitegrid')

In [ ]:
plt.plot(n_trials_list, test_metrics, '-o')

In [96]:
import shutil

def plot_results(x, y, model_name, do_zip=False, dpi=500, delete_and_rewrite=True):
    cur_X_test, cur_y_test, *_ = get_train_test(x, y)

    model = opener(model_name)
    cur_prediction = model.predict(cur_X_test)
    print('rmse = ', np.sqrt(mean_squared_error(cur_y_test, cur_prediction)))
    
    if delete_and_rewrite:
        os.makedirs(model_name, exist_ok=True)
    
    for i, item in enumerate(cur_X_test[::20]):
        fig = plt.figure(figsize=(5, 4))
        plt.plot(
            cur_X_test[i * 20:(i + 1) * 20, -1],
            cur_prediction[i * 20:(i + 1) * 20],
            '-o',
            label='predicted'
        )
        plt.plot(
            cur_X_test[i * 20:(i + 1) * 20, -1],
            cur_y_test[i * 20:(i + 1) * 20],
            '-o',
            label='real'
        )
        title = f'red {item[0]}, cal {item[1]}, alpha {int(item[2])}, fric {item[3]}'
        plt.title(title)
        plt.legend(loc='best')
        plt.xlabel('r, -')
        plt.ylabel('$\sigma$, MPa')
        # plt.show()
        plt.savefig(''.join((model_name, '/', title, '.png')), format='png', dpi=dpi)
        plt.close()
        # break
    if do_zip: shutil.make_archive(model_name, 'zip', model_name)

In [102]:
X_stress_components[0].max(axis=0)

array([ 0.3,  1. , 20. ,  0.1])

In [97]:
plot_results(
    X_stress_components[0],
    y_stress_components[0],
    'optuna_200_3cv_no_SS',
    do_zip=True
)

d:\учеба/optuna_200_3cv_no_SS.pkl
rmse =  1.376178095536843


In [98]:
plot_results(
    X_stress_components[0],
    y_stress_components[0],
    'optuna_200_3cv',
    do_zip=True
)

d:\учеба/optuna_200_3cv.pkl
rmse =  4.872450206028714
